<a href="https://colab.research.google.com/github/babi2707/Trabalho-PAI---Reconhecimento-Papanicolau/blob/main/algoritmo/backend/Untitled5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [104]:
# Célula 1:
import zipfile

caminho_pasta_compactada = '/content/sub_images.zip'
caminho_destino = '/content/sub_images'

with zipfile.ZipFile(caminho_pasta_compactada, 'r') as zip_ref:
    zip_ref.extractall(caminho_destino)


In [105]:
# Célula 2:
import os

base_dir = '/content/sub_images/sub_images'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Criar diretórios principais
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Criar subdiretórios para cada classe
classes = ['ASC-H', 'ASC-US', 'HSIL', 'LSIL', 'Negative for intraepithelial lesion', 'SCC']
for class_name in classes:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)


In [106]:
# # Célula 3.1:
# from sklearn.model_selection import train_test_split
# import os

# # Diretório onde estão os dados
# data_dir = '/content/sub_images/sub_images'

# # Lista para armazenar os nomes dos arquivos e suas respectivas classes
# file_names = []
# labels = []

# # Preencher as listas com os nomes dos arquivos e suas classes correspondentes
# for class_name in os.listdir(data_dir):
#     class_dir = os.path.join(data_dir, class_name)
#     for file_name in os.listdir(class_dir):
#         file_path = os.path.join(class_dir, file_name)
#         file_names.append(file_path)
#         labels.append(class_name)

# # Dividir os dados em treino e validação (80% treino, 20% validação)
# train_files, val_files, train_labels, val_labels = train_test_split(
#     file_names, labels, test_size=0.2, stratify=labels, random_state=42
# )

# # Dividir os dados de treino em treino e teste (80% treino, 20% teste)
# train_files, test_files, train_labels, test_labels = train_test_split(
#     train_files, train_labels, test_size=0.2, stratify=train_labels, random_state=42
# )

# # Verificar o balanceamento das classes nos conjuntos de treino e teste
# for dataset, dataset_name in zip([train_labels, test_labels], ['treino', 'teste']):
#     unique, counts = np.unique(dataset, return_counts=True)
#     print(f'Balanceamento das classes no conjunto de {dataset_name}:')
#     for class_name, count in zip(unique, counts):
#         print(f'Classe {class_name}: {count} amostras')

# # Agora você tem:
# # - train_files: lista de arquivos de treino
# # - train_labels: lista de rótulos correspondentes de treino
# # - val_files: lista de arquivos de validação
# # - val_labels: lista de rótulos correspondentes de validação
# # - test_files: lista de arquivos de teste
# # - test_labels: lista de rótulos correspondentes de teste



# #Resultados
# # Found 966 images belonging to 6 classes.
# # Found 685 images belonging to 6 classes.
# # Found 704 images belonging to 6 classes.
# # Epoch 1/10
# # 31/31 [==============================] - 38s 158ms/step - loss: 2.9636 - accuracy: 0.2122 - val_loss: 1.7655 - val_accuracy: 0.2015
# # Epoch 2/10
# # 31/31 [==============================] - 3s 103ms/step - loss: 2.4541 - accuracy: 0.2402 - val_loss: 1.7478 - val_accuracy: 0.1985
# # Epoch 3/10
# # 31/31 [==============================] - 5s 156ms/step - loss: 2.2608 - accuracy: 0.2474 - val_loss: 1.7609 - val_accuracy: 0.1985
# # Epoch 4/10
# # 31/31 [==============================] - 3s 102ms/step - loss: 2.0217 - accuracy: 0.2826 - val_loss: 1.8062 - val_accuracy: 0.1985
# # Epoch 5/10
# # 31/31 [==============================] - 3s 100ms/step - loss: 1.9638 - accuracy: 0.2681 - val_loss: 1.7999 - val_accuracy: 0.1985
# # Epoch 6/10
# # 31/31 [==============================] - 3s 110ms/step - loss: 1.8415 - accuracy: 0.2867 - val_loss: 1.7896 - val_accuracy: 0.2234
# # Epoch 7/10
# # 31/31 [==============================] - 4s 143ms/step - loss: 1.8139 - accuracy: 0.2681 - val_loss: 1.7969 - val_accuracy: 0.2146
# # Epoch 8/10
# # 31/31 [==============================] - 3s 102ms/step - loss: 1.7435 - accuracy: 0.3033 - val_loss: 1.8410 - val_accuracy: 0.2219
# # Epoch 9/10
# # 31/31 [==============================] - 3s 105ms/step - loss: 1.7050 - accuracy: 0.2878 - val_loss: 1.8719 - val_accuracy: 0.2161
# # Epoch 10/10
# # 31/31 [==============================] - 5s 156ms/step - loss: 1.6880 - accuracy: 0.3095 - val_loss: 1.8447 - val_accuracy: 0.2263
# # 31/31 [==============================] - 2s 51ms/step
# # 22/22 [==============================] - 1s 49ms/step
# # Validation Accuracy (SVM): 0.19416058394160585
# #                                      precision    recall  f1-score   support

# #                               ASC-H       0.11      0.08      0.09        60
# #                              ASC-US       0.24      0.32      0.27       138
# #                                HSIL       0.18      0.18      0.18       147
# #                                LSIL       0.16      0.24      0.19       136
# # Negative for intraepithelial lesion       0.25      0.16      0.19       144
# #                                 SCC       0.16      0.05      0.08        60

# #                            accuracy                           0.19       685
# #                           macro avg       0.18      0.17      0.17       685
# #                        weighted avg       0.19      0.19      0.19       685

Balanceamento das classes no conjunto de treino:
Classe ASC-H: 53 amostras
Classe ASC-US: 128 amostras
Classe HSIL: 128 amostras
Classe LSIL: 128 amostras
Classe Negative for intraepithelial lesion: 128 amostras
Classe SCC: 52 amostras
Classe test: 4 amostras
Classe train: 4 amostras
Classe val: 4 amostras
Balanceamento das classes no conjunto de teste:
Classe ASC-H: 14 amostras
Classe ASC-US: 32 amostras
Classe HSIL: 32 amostras
Classe LSIL: 32 amostras
Classe Negative for intraepithelial lesion: 32 amostras
Classe SCC: 13 amostras
Classe test: 1 amostras
Classe train: 1 amostras
Classe val: 1 amostras


In [107]:
# Célula 3.2:
from sklearn.model_selection import train_test_split
import os

# Diretório onde estão os dados
data_dir = '/content/sub_images/sub_images'

# Lista para armazenar os nomes dos arquivos e suas respectivas classes
file_names = []
labels = []

# Preencher as listas com os nomes dos arquivos e suas classes correspondentes
for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        file_names.append(file_path)
        labels.append(class_name)

# Dividir os dados em treino e teste (80% treino, 20% teste)
train_files, test_files, train_labels, test_labels = train_test_split(
    file_names, labels, test_size=0.2, stratify=labels, random_state=42
)

# Verificar o balanceamento das classes nos conjuntos de treino e teste
for dataset, dataset_name in zip([train_labels, test_labels], ['treino', 'teste']):
    unique, counts = np.unique(dataset, return_counts=True)
    print(f'Balanceamento das classes no conjunto de {dataset_name}:')
    for class_name, count in zip(unique, counts):
        print(f'Classe {class_name}: {count} amostras')

# Agora você tem:
# - train_files: lista de arquivos de treino
# - train_labels: lista de rótulos correspondentes de treino
# - test_files: lista de arquivos de teste
# - test_labels: lista de rótulos correspondentes de teste

#Resultados:
# Found 966 images belonging to 6 classes.
# Found 643 images belonging to 6 classes.
# Found 656 images belonging to 6 classes.
# Epoch 1/10
# 31/31 [==============================] - 36s 172ms/step - loss: 3.1935 - accuracy: 0.1801 - val_loss: 1.7733 - val_accuracy: 0.1991
# Epoch 2/10
# 31/31 [==============================] - 4s 144ms/step - loss: 2.3475 - accuracy: 0.2733 - val_loss: 1.7861 - val_accuracy: 0.1991
# Epoch 3/10
# 31/31 [==============================] - 3s 104ms/step - loss: 2.2693 - accuracy: 0.2516 - val_loss: 1.7892 - val_accuracy: 0.2053
# Epoch 4/10
# 31/31 [==============================] - 3s 100ms/step - loss: 2.0218 - accuracy: 0.2557 - val_loss: 1.7805 - val_accuracy: 0.1991
# Epoch 5/10
# 31/31 [==============================] - 4s 136ms/step - loss: 1.9547 - accuracy: 0.2743 - val_loss: 1.8247 - val_accuracy: 0.2053
# Epoch 6/10
# 31/31 [==============================] - 3s 102ms/step - loss: 1.8596 - accuracy: 0.2743 - val_loss: 1.8279 - val_accuracy: 0.1975
# Epoch 7/10
# 31/31 [==============================] - 3s 101ms/step - loss: 1.8265 - accuracy: 0.2619 - val_loss: 1.8671 - val_accuracy: 0.1991
# Epoch 8/10
# 31/31 [==============================] - 4s 137ms/step - loss: 1.7337 - accuracy: 0.2950 - val_loss: 1.9080 - val_accuracy: 0.2022
# Epoch 9/10
# 31/31 [==============================] - 3s 100ms/step - loss: 1.7119 - accuracy: 0.2992 - val_loss: 1.9506 - val_accuracy: 0.2115
# Epoch 10/10
# 31/31 [==============================] - 3s 101ms/step - loss: 1.6747 - accuracy: 0.3178 - val_loss: 2.0451 - val_accuracy: 0.1788
# 31/31 [==============================] - 2s 49ms/step
# 21/21 [==============================] - 1s 61ms/step
# Validation Accuracy (SVM): 0.19595645412130638
#                                      precision    recall  f1-score   support

#                               ASC-H       0.12      0.05      0.07        57
#                              ASC-US       0.20      0.28      0.24       131
#                                HSIL       0.16      0.20      0.18       136
#                                LSIL       0.23      0.23      0.23       128
# Negative for intraepithelial lesion       0.20      0.19      0.20       132
#                                 SCC       0.22      0.07      0.10        59

#                            accuracy                           0.20       643
#                           macro avg       0.19      0.17      0.17       643
#                        weighted avg       0.20      0.20      0.19       643


In [108]:
# Célula 4:
import shutil
import random

def split_data(source_dir, train_dir, val_dir, test_dir, split_ratio=(0.7, 0.15, 0.15)):
    for class_name in classes:
        class_dir = os.path.join(source_dir, class_name)
        images = os.listdir(class_dir)
        random.shuffle(images)

        train_split = int(split_ratio[0] * len(images))
        val_split = int((split_ratio[0] + split_ratio[1]) * len(images))

        train_images = images[:train_split]
        val_images = images[train_split:val_split]
        test_images = images[val_split:]

        for image in train_images:
            shutil.move(os.path.join(class_dir, image), os.path.join(train_dir, class_name, image))

        for image in val_images:
            shutil.move(os.path.join(class_dir, image), os.path.join(val_dir, class_name, image))

        for image in test_images:
            shutil.move(os.path.join(class_dir, image), os.path.join(test_dir, class_name, image))

split_data(base_dir, train_dir, val_dir, test_dir)


In [109]:
# Célula 5:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Definir os caminhos para os dados de treinamento e validação
train_dir = '/content/sub_images/sub_images/train'
val_dir = '/content/sub_images/sub_images/val'
test_dir = '/content/sub_images/sub_images/test'

# Parâmetros
img_height, img_width = 32, 32
batch_size = 32
num_classes = 6
class_names = ['ASC-H', 'ASC-US', 'HSIL', 'LSIL', 'Negative for intraepithelial lesion', 'SCC']

# Criar um gerador de dados para carregar e pré-processar as imagens
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Carregar os dados de treinamento
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Carregar os dados de validação
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Carregar os dados de teste
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)
# Carregar a ResNet50 pré-treinada
base_model = ResNet50(weights=None, include_top=False, input_shape=(img_height, img_width, 3))

# Adicionar camadas ao modelo
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Adicionando dropout
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar o modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),  # Ajustando a taxa de aprendizado
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=val_generator,
    validation_steps=len(val_generator),
    epochs=10
)

# Extrair características para treinamento e validação
train_features = model.predict(train_generator)
val_features = model.predict(val_generator)

# Converter rótulos para uma única classe (não one-hot encoding)
train_labels = train_generator.classes
val_labels = val_generator.classes

# Treinar o classificador SVM com hiperparâmetros ajustados
svm = make_pipeline(StandardScaler(), SVC(kernel='rbf', C=100, gamma=0.1, probability=True))  # Ajustando os hiperparâmetros
svm.fit(train_features, train_labels)

# Avaliar o classificador SVM
val_predictions = svm.predict(val_features)
val_accuracy = accuracy_score(val_labels, val_predictions)
print('Validation Accuracy (SVM):', val_accuracy)
print(classification_report(val_labels, val_predictions, target_names=class_names))


Found 966 images belonging to 6 classes.
Found 685 images belonging to 6 classes.
Found 704 images belonging to 6 classes.
Epoch 1/10
31/31 [==============================] - 38s 158ms/step - loss: 2.9636 - accuracy: 0.2122 - val_loss: 1.7655 - val_accuracy: 0.2015
Epoch 2/10
31/31 [==============================] - 3s 103ms/step - loss: 2.4541 - accuracy: 0.2402 - val_loss: 1.7478 - val_accuracy: 0.1985
Epoch 3/10
31/31 [==============================] - 5s 156ms/step - loss: 2.2608 - accuracy: 0.2474 - val_loss: 1.7609 - val_accuracy: 0.1985
Epoch 4/10
31/31 [==============================] - 3s 102ms/step - loss: 2.0217 - accuracy: 0.2826 - val_loss: 1.8062 - val_accuracy: 0.1985
Epoch 5/10
31/31 [==============================] - 3s 100ms/step - loss: 1.9638 - accuracy: 0.2681 - val_loss: 1.7999 - val_accuracy: 0.1985
Epoch 6/10
31/31 [==============================] - 3s 110ms/step - loss: 1.8415 - accuracy: 0.2867 - val_loss: 1.7896 - val_accuracy: 0.2234
Epoch 7/10
31/31 [======